## COMP90042 Project 2022: Rumour Detection and Analysis on Twitter

# Task 1

## 1. Importing Libraries and API

In [ ]:
## API needed for tweet retrieving

import tweepy
import time
import json
import os
import re
import csv
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

## Twitter API
yizhou_API_KEY = ""
yizhou_API_KEY_SECRET = ""
yizhou_ACCESS_TOKEN = ""
yizhou_ACCESS_TOKEN_SECRET = ""

## 2. Retriving Tweets

In [ ]:
## get train_data tweet object

## get tweet object with tweet ID via API

## access API
auth = tweepy.OAuthHandler(yizhou_API_KEY, yizhou_API_KEY_SECRET)
auth.set_access_token(yizhou_ACCESS_TOKEN, yizhou_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

# ## test if the API works
trainID = "1250219300389974016"
tweetFetched = api.get_status(trainID)

## read train_tweetID
train_raw = open("train.data.txt", "r")
train_event_list = []
for event in train_raw:
    event = event[:-1] ## get rid of \n in each line
    train_event_list.append(event)

## write fetched tweet_object in JSON
tweet_with_issue = {}
for event in train_event_list:
    tweet_list = event.split(",")
    for tweet in tweet_list:
        if (not os.path.isfile("./train_data/" + tweet + ".json")):
            try:
                tweet_object = api.get_status(tweet)._json
                tweet_object_string = json.dumps(tweet_object)
                with open("train_data/" + tweet+".json", "w") as outfile:
                    json.dump(tweet_object_string, outfile)
            except tweepy.errors.NotFound:
                tweet_with_issue[tweet] = "tweet not found"
                print("not found")
            except tweepy.errors.Forbidden:
                tweet_with_issue[tweet] = "user suspended"
                print("user suspended")
            except Exception as e:
                tweet_with_issue[tweet] = "other issue"
                print(e)


with open('tweet_with_issue.txt', 'w') as twi:
    twi.write(str(tweet_with_issue))

tweet_with_issue = open("tweet_with_issue.txt", "r")
tweet_dictionary = json.loads(tweet_with_issue.read().replace("\'", "\""))

for tweet in tweet_dictionary:
    if tweet_dictionary[tweet] == "other issue":
        tweet_object = api.get_status(tweet)._json
        tweet_object_string = json.dumps(tweet_object)
        with open("train_data/" + tweet+".json", "w") as outfile:
            json.dump(tweet_object_string, outfile)


### Retrieve json needed for task 2.

## read train_tweetID
covid_raw = open("covid.data.txt", "r")
line = 0
covid_event_list = []
for event in covid_raw:
    event = event[:-1] ## get rid of \n in each line
    covid_event_list.append(event)
    line += 1

## write fetched tweet_object in JSON
tweet_count = 0
tweet_with_issue = {}
for i in range(8735):
    tweet_list = covid_event_list[i].split(",")
    for tweet in tweet_list:
        tweet_count += 1
        if (not os.path.isfile("./covid_data/" + tweet + ".json")):
            try:
                tweet_object = api.get_status(tweet)._json
                tweet_object_string = json.dumps(tweet_object)
                with open("covid_data/" + tweet+".json", "w") as outfile:
                    json.dump(tweet_object_string, outfile)
                    print(str(float(tweet_count/126574)), tweet, "success")
            except tweepy.errors.NotFound:
                tweet_with_issue[tweet] = "tweet not found"
                print(str(float(tweet_count/126574)), tweet, "not found")
            except tweepy.errors.Forbidden:
                tweet_with_issue[tweet] = "user suspended"
                print(str(float(tweet_count/126574)), tweet, "user suspended")
            except Exception as e:
                tweet_with_issue[tweet] = "other issue"
                print(str(float(tweet_count/126574)), tweet, str(e))


with open('covid_with_issue.txt', 'w') as twi:
    twi.write(str(tweet_with_issue))


# to retrieve again the tweets with internet issue last time

covid_dictionary = open("covid_with_issue.txt", "r")
dict_str = covid_dictionary.readline()
dict_str = dict_str.replace("\'", "\"")
res = json.loads(dict_str)

count = 0
for tweet in res:
    if res[tweet] == "other issue":
        try:
            tweet_object = api.get_status(tweet)._json
            tweet_object_string = json.dumps(tweet_object)
            with open("covid_data/" + tweet+".json", "w") as outfile:
                json.dump(tweet_object_string, outfile)
        except tweepy.errors.NotFound:
            print("not found")
        except tweepy.errors.Forbidden:
            print("user suspended")

        except Exception as e:
            print("other issue")


## 3. Data Preprocessing

In [ ]:
# we need to skip the events with empty source tweet (user suspended/ tweet not found)
index_with_empty_tweet_train = set()
index_with_empty_tweet_dev = set()
index_with_empty_tweet_covid = set()

train_raw = open("train.data.txt", "r")
issue_index_train = 0
for event in train_raw:
    tweets = event[:-1].split(',')
    if (not os.path.isfile("./train_data/" + tweets[0] + ".json")):
        index_with_empty_tweet_train.add(issue_index_train)
    issue_index_train += 1
dev_raw = open("dev.data.txt", "r")
issue_index_dev = 0
for event in dev_raw:
    tweets = event[:-1].split(',')
    if (not os.path.isfile("./dev_data/" + tweets[0] + ".json")):
        index_with_empty_tweet_dev.add(issue_index_dev)
    issue_index_dev += 1

covid_raw = open("covid.data.txt", "r")
issue_index_covid = 0
for event in covid_raw:
    tweets = event[:-1].split(',')
    if (not os.path.isfile("./covid_data/" + tweets[0] + ".json")):
        index_with_empty_tweet_covid.add(issue_index_covid)
    issue_index_covid += 1
print(len(index_with_empty_tweet_train))
print(len(index_with_empty_tweet_dev))

def getJSONwithID(tweet_id, path):
    str_tweet_json = ""
    if (not os.path.isfile("./" + path + "/" + tweet_id + ".json")):
        return {}
    with open(path + "/" + tweet_id + ".json", "r") as file:
        if path == "tweet-objects":
            return json.loads(file.readline())
        else:
            str_tweet_json = file.readline()
    file.close()
    str_tweet_json = str_tweet_json.replace('\\"',"\"").replace('\\\\', "\\")[1:-1]
    obj = json.loads(str_tweet_json)

        
    return obj

def sortTweetsByTime(tweet_list, path):
    unsorted_tweet_list = []
    for tweet_ID in tweet_list:
        tweet_json = getJSONwithID(tweet_ID, path)
        if "created_at" in tweet_json:
            formatted_created_at = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet_json["created_at"], '%a %b %d %H:%M:%S +0000 %Y'))
            unsorted_tweet_list.append((tweet_ID, formatted_created_at))
    sorted_tweet_list_tuple = sorted(unsorted_tweet_list, key=lambda x: x[1])
    sorted_tweet_list = []
    for tweet_id, formatted_created_at in sorted_tweet_list_tuple:
        sorted_tweet_list.append(tweet_id)
    return sorted_tweet_list
            
# featureONE: only the source tweet text
def getFeatureONE(filename, path, funcList, issue_index):
    source_tweet_list = []
    file = open(filename+".txt", "r")## train.data
    index = 0
    for event in file:
        if index in issue_index:
            index += 1
            continue
        event = event[:-2]
        tweet_list = event.split(',')
        source_tweet_id = tweet_list[0]
        source_tweet_json = getJSONwithID(source_tweet_id, path)
        if "text" not in source_tweet_json:
            source_tweet_text = ""
        else:
            source_tweet_text = source_tweet_json["text"]
        source_tweet_list.append(engineerTweet(source_tweet_text, funcList))
        index += 1
    file.close()
    return source_tweet_list

# featureTWO: all tweets in the event
def getFeatureTWO(filename, path, funcList, issue_index):
    tweets_list = []
    file = open(filename+".txt", "r")
    index = 0
    for event in file:
        if index in issue_index:
            index += 1
            continue
        event = event[:-1]
        tweet_txt_list = []
        tweet_list = event.split(',')
        for tweet_id in tweet_list:
            tweet_json = getJSONwithID(tweet_id, path)
            if "text" not in tweet_json:
                continue
            else:
                tweet_text = tweet_json["text"]
            tweet_txt_list.append(engineerTweet(tweet_text, funcList))
        tweets_list.append(tweet_txt_list)
        index += 1
    file.close()
    return tweets_list

# featureTHREE: get source tweet and the last x replies in the event
def getFeatureTHREE(filename, path, noOfReplies, funcList, issue_index):
    tweets_list = []
    file = open(filename+".txt", "r")
    index = 0
    for event in file:
        if index in issue_index:
            index += 1
            continue
        tweet_txt_list = []
        event = event[:-1]
        tweet_list = event.split(',')
        source_tweet_id = tweet_list[0]
        source_tweet_json = getJSONwithID(source_tweet_id, path)
        if "text" not in source_tweet_json:
            tweet_txt_list.append("")
        else:
            tweet_txt_list.append(engineerTweet(source_tweet_json["text"], funcList))
        if len(tweet_list) > 1:
            reply_tweet_id_list = tweet_list[1:]
            sorted_reply_tweet_id_list = sortTweetsByTime(reply_tweet_id_list, path)
            if len(sorted_reply_tweet_id_list) > noOfReplies:
                sorted_reply_tweet_id_list = sorted_reply_tweet_id_list[-1 * noOfReplies: ]
            for tweet_id in sorted_reply_tweet_id_list:
                reply_txt = engineerTweet(getJSONwithID(tweet_id, path)["text"], funcList)
                tweet_txt_list.append(reply_txt)
        tweets_list.append(tweet_txt_list)
        index += 1
    return tweets_list



def engineerTweet(tweet, functionList):
    for func in functionList:
        tweet = func(tweet)
    return tweet

## feature engineering
def defaultEngineering(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r' {2,}',' ', tweet)
    return tweet.replace("\n", " ")

def removeStopWords(tweet):
    tweet_list = tweet.split()
    return ' '.join([i for i in tweet_list if i not in stop_words])

def removeURL(tweet):
    return re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', 'HTTP', tweet)

def removeAtMentions(tweet):
    return re.sub(r'@[A-Za-z0-9_]+', '@USER', tweet)

def removePunctuationMarks(tweet):
    return re.sub(r'[^a-zA-Z#]', ' ', tweet)

def shortenDuplicatedLetters(tweet):
    return re.sub(r'([a-z])\1+', r'\1'r'\1', tweet) # replace 'coool' with 'cool' (more than 3 repeats to 2)

def removeNonAlphaNumericCharacters(tweet):
    return re.sub("[^a-z0-9]"," ", tweet)

def removeDuplicatedSpace(tweet):
    tweet = re.sub(r' {2,}',' ', tweet)
    if tweet == "":
        return tweet
    if tweet[0] == " ":
        tweet = tweet[1:]
    if tweet == "":
        return tweet
    if tweet[-1] == " ":
        tweet = tweet[:-1]
    return tweet
    

origin_tweet = "Hao RT @WSJ: Breaking: Microsoft is near a deal to buy Minecraft maker Mojang  http://t.co/2S6t0Hvy28"
processed_tweet = engineerTweet(origin_tweet, [defaultEngineering, removeStopWords, removeAtMentions, removeURL, removePunctuationMarks, shortenDuplicatedLetters])
print(processed_tweet)

def getLabel(filename, issue_index):
    label_list = []
    file = open(filename + ".txt", "r")
    index = 0
    for label in file:
        if index in issue_index:
            index += 1
            continue
        label_list.append(label[:-1])
        index += 1
    return label_list


def writeToTxt(string_list, filename):
    with open("./data_txt/" + filename + ".txt", 'w') as f:
        for string in string_list:
            f.write(string)
            f.write('\n')
        f.close()

def writeToTSV(X, y, filename):
    with open(filename + ".tsv", "wt") as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        tsv_writer.writerow(['sentence', 'label'])
        for i in range(len(X)):
            if y[i] == "rumour":
                tsv_writer.writerow([X[i], 1])
            else:
                tsv_writer.writerow([X[i], 0])
    out_file.close()

def twoDListToList(li):
    output_list = []
    for i in range(len(li)):
        long_string = listToString(li[i])
        output_list.append(long_string)
    return output_list


def listToString(li):
    long_string = " [SEP] "
    return (long_string.join(li))

    
funcList = [removeAtMentions, removeURL]

X_train_feature1 = getFeatureONE("train.data", "train_data", funcList, index_with_empty_tweet_train)
X_train_feature2 = getFeatureTWO("train.data", "train_data", funcList, index_with_empty_tweet_train)
X_train_feature3 = getFeatureTHREE("train.data", "train_data", 500, funcList, index_with_empty_tweet_train)
y_train = getLabel("train.label", index_with_empty_tweet_train)

X_train_feature2_x = twoDListToList(X_train_feature2)
X_train_feature3_x = twoDListToList(X_train_feature3)

X_dev_feature1 = getFeatureONE("dev.data", "dev_data", funcList, index_with_empty_tweet_dev)
X_dev_feature2 = getFeatureTWO("dev.data", "dev_data", funcList, index_with_empty_tweet_dev)
X_dev_feature3 = getFeatureTHREE("dev.data", "dev_data", 500, funcList, index_with_empty_tweet_dev)
y_dev = getLabel("dev.label", index_with_empty_tweet_dev)


X_covid_feature3 = getFeatureTHREE("covid.data", "covid_data", 500, funcList, index_with_empty_tweet_covid)
y_covid = [99] * len(X_covid_feature3)

X_covid_feature3_x = twoDListToList(X_covid_feature3)
writeToTSV(X_covid_feature3_x, y_covid, "feature3_covid")



y_dev_1 = []
for y in y_dev:
    if y == "nonrumour":
        y_dev_1.append(0)
    else:
        y_dev_1.append(1)
print(len(y_dev_1))
print(y_dev_1)

X_dev_feature2_x = twoDListToList(X_dev_feature2)
X_dev_feature3_x = twoDListToList(X_dev_feature3)

X_test_feature1 = getFeatureONE("test.data", "tweet-objects", funcList, set())
X_test_feature2 = getFeatureTWO("test.data", "tweet-objects", funcList, set())
X_test_feature3 = getFeatureTHREE("test.data", "tweet-objects", 500, funcList, set())

X_test_feature2_x = twoDListToList(X_test_feature2)
X_test_feature3_x = twoDListToList(X_test_feature3)
y_test = [99] * len(X_test_feature1)




writeToTSV(X_train_feature1, y_train, "feature1_train")
writeToTSV(X_train_feature2_x, y_train, "feature2_train")
writeToTSV(X_train_feature3_x, y_train, "feature3_train")
writeToTSV(X_dev_feature1, y_dev, "feature1_dev")
writeToTSV(X_dev_feature2_x, y_dev, "feature2_dev")
writeToTSV(X_dev_feature3_x, y_dev, "feature3_dev")
writeToTSV(X_test_feature1, y_test, "feature1_test")
writeToTSV(X_test_feature2_x, y_test, "feature2_test")
writeToTSV(X_test_feature3_x, y_test, "feature3_test")
assert(len(X_train_feature1) == 1895 - 330)
assert(len(X_train_feature2) == 1895 - 330)
assert(len(X_train_feature3) == 1895 - 330)
assert(len(y_train) == 1895 - 330)
assert(len(X_dev_feature1) == 632 - 95)
assert(len(X_dev_feature2) == 632 - 95)
assert(len(X_dev_feature3) == 632 - 95)
assert(len(y_dev) == 632 - 95)

## Dataset



## initializer 

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import Dataset
class sstdataset(Dataset):
    def __init__(self, maxlen,tokernizertype ='bert-base-uncased'):
        self.maxlen = maxlen
        self.tokenizer = BertTokenizer.from_pretrained(tokernizertype,
        num_labels = 2
        )
        self.ATTNMASK_ARRAY = []
        self.TOKEN_IDS = []
        self.LABEL = []

    def bert_sentences_processing(self,sentences,label):
        SegID = 0
        token = ['[CLS]']
        SegIDs = []
        token_len = 1
        for sentence in sentences:
            current_token = self.tokenizer.tokenize(sentence)
            len_of_current_token = len(current_token)
            token_len += (len_of_current_token + 1)
            token += current_token + ['[SEP]']
            SegIDs.extend((len_of_current_token + 1 ) * [SegID]) 
            SegID += 1
        if int(token_len) < int(self.maxlen):
            token = token + ['[PAD]' for _ in range(self.maxlen - token_len)]
        else:
            if token[self.maxlen-2] == '[SEP]':
                token = token[:self.maxlen-1] + ['[PAD]']
            else:
                token = token[:self.maxlen-1] + ['[SEP]']
        token_ids = self.tokenizer.convert_tokens_to_ids(token)
        attn_mask = [int(num != 0) for num in token_ids]
        self.TOKEN_IDS.append(token_ids)
        self.ATTNMASK_ARRAY.append(attn_mask)
        self.LABEL.append(label)
    def __len__(self):
        return len(self.LABEL)
    def __getitem__(self,index):
        return torch.tensor(self.TOKEN_IDS[index]), torch.tensor(self.ATTNMASK_ARRAY[index]),torch.tensor(self.LABEL[index])

## Constructing dataset for test, train, and dev

In [ ]:
from torch.utils.data import DataLoader
#paramter for dataloaders
DATABASESIZE = 130
BATCHSIZE = 1
NUMBEROFWORKER = 0

# construction
BERTtrainDataset = sstdataset(DATABASESIZE)
BertDEVDataset = sstdataset(DATABASESIZE)
BertOverallDataset = sstdataset(DATABASESIZE)
BertTESTDataset = sstdataset(DATABASESIZE)

# loading from the preprocessed text

#adding training set
for addxtrain, addytrain in zip(X_train_feature3,y_train):
    if(addxtrain):
        BERTtrainDataset.bert_sentences_processing(addxtrain,addytrain)
#adding testing data
for addxtrain, addytrain in zip(X_dev_feature3,y_dev):
    if(addxtrain):
        BertDEVDataset.bert_sentences_processing(addxtrain,addytrain)
# loading the test dataset
yyindex = 0
for addxtest in X_test_feature3:
    BertTESTDataset.bert_sentences_processing(addxtest,yyindex)
    yyindex += 1
    

#create data loader for the dataset objects
train_loader = DataLoader(BERTtrainDataset, batch_size = BATCHSIZE, num_workers = NUMBEROFWORKER)
dev_loader = DataLoader(BertDEVDataset, batch_size = BATCHSIZE, num_workers = NUMBEROFWORKER)
test_loader = DataLoader(BertTESTDataset, batch_size = BATCHSIZE, num_workers = NUMBEROFWORKER)

## Models

### BERT_FNN_Single Layer

In [ ]:
from torch.utils.data import Dataset
from transformers import BertTokenizer
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertModel

class SentimentClassifier(nn.Module):

    def __init__(self):
        super(SentimentClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        self.cls_layer = nn.Linear(768, 1)

    def forward(self, seq, attn_masks):
        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert_layer(seq, attention_mask = attn_masks)
        cont_reps = outputs.last_hidden_state

        # #Obtaining the representation of [CLS] head (the first token)
        cls_rep = cont_reps[:, 0]

        # #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(cls_rep)
        return logits

gpu = 0 #gpu ID
net = SentimentClassifier()
net.cuda(gpu) 




criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)


import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), 'sstcls_{}.dat'.format(ep))

def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count


num_epoch = 10

#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)


from sklearn.metrics import classification_report, confusion_matrix

def get_predictions(model, loader):
    model = model.eval()
    predictions = []
    probs_all = []
    with torch.no_grad():
        for seq, attn_masks, label in loader:
            seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
            logits = model(seq, attn_masks)
            probs = torch.sigmoid(logits).cpu().numpy().flatten()
            prob = [prob for prob in probs]
            probs_all += prob
            soft_probs = [int(prob > 0.5) for prob in probs]
            predictions += soft_probs
    return [predictions, probs_all]

import csv
filename = "covid_results.csv"
#y_dev = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0]
pre = get_predictions(net,covid_loader)
with open(filename,'w',newline='') as f:
    csvwritter = csv.writer(f,delimiter = ',')
    csvwritter.writerow(["Id","Predicted","prob"])
    for i in range(len(pre[0])):
      csvwritter.writerow([i, pre[0][i], pre[1][i]])

# pre = get_predictions(net,dev_loader)
# y_dev = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0]
# print(classification_report(y_dev, pre, digits=5))



### Multi-layers FNN with BERT

In [ ]:
################################### MODLE CONSTRUCTION #################################
import torch.nn.functional as F
from transformers import BertModel, get_linear_schedule_with_warmup
import torch.optim as optim
import torch.nn as nn
import time
import torch
class FeedforwardNeuralNetModel(nn.Module):
  def __init__(self):
    super(FeedforwardNeuralNetModel, self).__init__()
    HIDDENDIMENTION = 256
    self.bert_layer = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
    self.fc1 = nn.Linear(768, HIDDENDIMENTION)
    self.fc2 = nn.Linear(HIDDENDIMENTION,1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, seq, attn_masks):
        # Linear function  # LINEAR
      outputs = self.bert_layer(seq, attention_mask = attn_masks, return_dict=True)
      cont_reps = outputs.last_hidden_state
      cls_rep = cont_reps[:, 0]
      # into hidden layers
      x = F.relu(self.fc1(cls_rep))
      x = self.fc2(x)
      x = self.sigmoid(x)
      return x

FNNgpu = 0
FNNnet = FeedforwardNeuralNetModel()
FNNnet.cuda(FNNgpu)
FNNcriterion =  nn.BCELoss()
FNNoptimizer = torch.optim.AdamW(FNNnet.parameters(), lr=2e-5, weight_decay=0.97)

#################################### training ##############################################
def FNNtrain(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):
    best_acc = 0
    st = time.time()
    total_steps = len(train_loader) * max_eps
    for ep in range(max_eps):
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq = seq.cuda(gpu)
            attn_masks = attn_masks.cuda(gpu)
            labels = labels.cuda(gpu)
            logits = net(seq, attn_masks)
            loss = criterion(logits, labels.unsqueeze(1).float())
            loss.backward()
            opti.step()
            
            if it % 200 == 0:
                print("Iteration {} of epoch {} Time taken (s): {}, loss =  {}".format(it, ep, (time.time()-st),loss.item()))
                st = time.time() 


In [ ]:
# fine- tuning
FNNtrain(FNNnet, FNNcriterion, FNNoptimizer, overall_loader, dev_loader, 3, FNNgpu)  

### Logestic Regression Classifier with last 4 layers

In [ ]:
from transformers import BertModel
import torch.optim as optim
import torch.nn as nn
import time
import torch
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased',
        num_labels = 2
        )
        #768 for base; 1024 for large
        self.lin = nn.Linear(3072, 1)


    def forward(self, seq, attn_masks):        
        cls_hs  = self.bert_layer(seq, attention_mask = attn_masks, return_dict= True,  output_hidden_states=True)
        encoded_layers = cls_hs['hidden_states'][-4:]
        concatenated = torch.cat(encoded_layers, -1)
        logits = self.lin(concatenated[:, 0])
        
        return logits

# assinging a gpu to the model 
gpu = 0
net = LogisticRegression()
net.cuda(gpu)
criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)



import torch
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

    import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq = seq.cuda(gpu)
            attn_masks = attn_masks.cuda(gpu)
            labels = labels.cuda(gpu)
            logits = net(seq, attn_masks)
            loss = criterion(logits.squeeze(-1), labels.float())
            loss.backward()
            opti.step()
            if it % 100 == 0:
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), 'sstcls_{}.dat'.format(ep))
num_epoch = 11

#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

### Convulution Classifier with last 4 bert layers


In [ ]:
import torch.nn.functional as F
from transformers import BertModel, get_linear_schedule_with_warmup
import torch.optim as optim
import torch.nn as nn
import time
import torch
#loss_fn = nn.CrossEntropyLoss()
class CNNBert(nn.Module):

    def __init__(self, embed_size, bert_model='bert-base-uncased'):
        super(CNNBert, self).__init__()
        self.bert_layer = BertModel.from_pretrained(bert_model, output_hidden_states=True
        )
        filter_sizes = [1,2,3,4,5]
        num_filters = 32
        self.convs1 = nn.ModuleList([nn.Conv2d(4, num_filters, (K, embed_size)) for K in filter_sizes])
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(len(filter_sizes)*num_filters, 1)
        self.sigmoid = nn.Sigmoid()
        self.bert_model = bert_model

    def forward(self, seq, attn_masks):   
        x = self.bert_layer(seq, attention_mask = attn_masks, output_hidden_states=True)[2][-4:]
        x = torch.stack(x, dim=1)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1] 
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        x = torch.cat(x, 1)
        logit = self.fc1(x)
        
        return self.sigmoid(logit)
CNNgpu = 0
CNNnet = CNNBert(768)
CNNnet.to(CNNgpu)   
CNNopti = torch.optim.AdamW(CNNnet.parameters(), lr=2e-5, weight_decay=0.97)
CNNcriterion = nn.BCELoss()

In [ ]:
import time
def CNNtrain(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):
    best_acc = 0
    st = time.time()

    total_steps = len(train_loader) * max_eps
    scheduler = get_linear_schedule_with_warmup(opti, 
                                        num_warmup_steps = 0,
                                        num_training_steps = total_steps)
    for ep in range(max_eps):
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq = seq.cuda(gpu)
            attn_masks = attn_masks.cuda(gpu)
            labels = labels.cuda(gpu)
            logits = net(seq, attn_masks)
            loss = criterion(logits, labels.unsqueeze(1).float())
            loss.backward()
            opti.step()
            
            if it % 200 == 0:
                print("Iteration {} of epoch {} Time taken (s): {}, loss =  {}".format(it, ep, (time.time()-st),loss.item()))
                st = time.time() 

CNNtrain(CNNnet, CNNcriterion, CNNopti, overall_loader, dev_loader, 4, CNNgpu)            

### LSTM / LSTM with BERT

In [ ]:
import tensorflow as tf
from keras import layers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.layers import LSTM

In [ ]:
tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(X_train_feature3)

y_train = np.array(y_train, dtype=np.int32)
y_dev = np.array(y_dev, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)

x_train = tokenizer.texts_to_matrix(X_train_feature3, mode="count")

vocab_size = x_train.shape[1]

x_train = tokenizer.texts_to_sequences(X_train_feature3)
x_dev = tokenizer.texts_to_sequences(X_dev_feature3)
x_test = tokenizer.texts_to_sequences(X_test_feature3)

maxlen = 30
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_dev = pad_sequences(x_dev, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

print("Vocab size =", vocab_size)
print(vocab_size)
print(len(y_train))

embedding_dim = 20


In [ ]:
# pure LSTM
model = Sequential(name="lstm")
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(LSTM(10))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

model.fit(x_train, y_train, epochs=1, batch_size=2)

loss, accuracy = model.evaluate(x_dev, y_dev, verbose=False)

print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, bias=True):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
        # Number of hidden layers
        self.num_layers = num_layers

        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=False, bidirectional=True)
        self.sigmoid = nn.Sigmoid()
        self.fc = nn.Linear(hidden_dim,1)
    
    def forward(self, seq, attn_masks):
        bert_output = self.bert_layer(seq, attention_mask = attn_masks, return_dict=True)
        cont_reps = bert_output.last_hidden_state
        lstm_output, _ = self.lstm(cont_reps)
        lstm_out = lstm_output.contiguous().view(-1, self.hidden_dim)
        out = self.fc(lstm_out)
        out = self.sigmoid(out)
        out = out.view(BATCHSIZE, -1)
        out = out[:,-1]
        return out

In [ ]:
input_dim = 768
hidden_dim = 100
num_layers = 2
output_dim = 1
 
model = LSTM(input_dim, hidden_dim, num_layers, output_dim)

if torch.cuda.is_available():
    model.cuda()

criterion = nn.BCEWithLogitsLoss()
lstmGPU = 0
optimizer = optim.Adam(model.parameters(), lr=2e-5)
train(model, criterion, optimizer, train_loader, dev_loader, 4, lstmGPU)

## evaluation 

In [ ]:
def get_predictions(model, loader):
    model = model.eval()
    predictions = []
    with torch.no_grad():
        for seq, attn_masks, label in loader:
            seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
            logits = model(seq, attn_masks)
            probs = logits.cpu().numpy().flatten()
            soft_probs = [int(prob > 0.5) for prob in probs]
            predictions += soft_probs
    return predictions

def Evaluationfile(model,dev_loader):
    pre = get_predictions(model,dev_loader)
    with open("my_dev_pre_lable.txt",'w',newline='') as f:
        csvwritter = csv.writer(f,delimiter = ',')
        for prediction in pre:
            csvwritter.writerow(["rumour"]) if prediction == 1 else csvwritter.writerow(["nonrumour"])
    
    acutallabels = BertDEVDataset.LABEL
    with open("my_dev_actual_lable.txt",'w',newline='') as f2:
        csvwritter = csv.writer(f2,delimiter = ',')
        for acutallabel in acutallabels:
            csvwritter.writerow(["rumour"]) if acutallabel == 1 else csvwritter.writerow(["nonrumour"])